In [1]:
import os
import google.genai as genai
from dotenv import load_dotenv

In [2]:
# .env 로드 및 api키 전역에 설정.
load_dotenv() 
google_key = os.getenv("GEMINI_API_KEY")

MODEL = "model/gemini-2.5-flash"

In [ ]:
FILE_NAME = "data_1.xlsx"

In [4]:
# 파일 import 및 저장 경로 설정
from pathlib import Path

PROJECT_ROOT = Path.cwd()
if PROJECT_ROOT.name == "notebooks":
    PROJECT_ROOT = PROJECT_ROOT.parent

DATA_PATH = PROJECT_ROOT / "downloads" / FILE_NAME
MD_PATH = PROJECT_ROOT / "debug" / "parsed_table.md"

In [5]:
# excel or csv의 raw data를 markdown으로 파싱

import importlib
import utils.parsing as parsing

importlib.reload(parsing)

from utils.parsing import parse_workbook_all_sheets_to_markdown


md_all = parse_workbook_all_sheets_to_markdown(
    DATA_PATH,
    max_total_chars=300_000,
    max_rows_per_sheet=2000,
    save_combined_markdown=False,
    combined_markdown_path=MD_PATH,
)


In [6]:
# 파싱된 마크다운 확인
md_all

'## Sheet: 폐강확정강좌(게시용)\n\n|   폐강기준 인원 |   수강인원 (대학원포함) | 승인원  검토 결과   |   학년 | 과정구분   |   수강번호 | 과목명                     |   학점 | 담당교수                  | 개설대학   | 개설학과   | 주관대학     | 주관학과   | 수업형태   | 비고                                   | 강의타입   |\n|----------------:|------------------------:|:--------------------|-------:|:-----------|-----------:|:---------------------------|-------:|:--------------------------|:-----------|:-----------|:-------------|:-----------|:-----------|:---------------------------------------|:-----------|\n|              60 |                      30 | 폐강                |      1 | 일반선택   |        319 | 소통-행복과변화로가는길    |      3 | 이윤주                    | 대학       | 대학       | 사범대학     | 교육학과   | 인터넷강의 | 인터넷강의                             | 일반강의   |\n|              20 |                      11 | 폐강                |      2 | 교직       |        363 | 교육학개론                 |      2 | 이상린                    | 대학       | 대학       | 사범대학     | 교육학과   | nan        | na

In [7]:
# load prompts
from utils.prompt_loader import load_json_generator_prompts, load_report_generator_prompts

json_generator_prompts = load_json_generator_prompts()
report_generator_prompts = load_report_generator_prompts()

/Users/anseonghui/text-to-json/prompt


In [8]:
json_generator_prompts

'You will be given structured extracts derived from a source document.\nYour task is to produce two artifacts from this information:\n\n1. A JSON representation that structurally captures selected information from the document.\n2. A JSON Schema that describes exactly and only the structure of the generated JSON.\n\nIMPORTANT WORLDVIEW CONSTRAINTS:\n\n* You must NOT assume that the source data comes from spreadsheets, tables, cells, or files of any kind.\n* The JSON must be fully derivable from the document text alone.\n* The JSON Schema must be specific to this JSON output and MAY differ for every document.\n* If a concept does not explicitly appear in the document, it must NOT appear in the JSON or the JSON Schema.\n* Do NOT introduce new facts, interpretations, entities, inferred classifications, or additional categories beyond what is explicitly stated.\n* If information is missing, do not fabricate values. Omit the field entirely unless the absence itself is explicitly stated in t

In [9]:
report_generator_prompts

'You are given an Markdown that parsed from Excel spreadsheet.\n\nYour task is to write a **professional analytical report** grounded in the data, written as a coherent standalone document.\n\nYou **must include a placeholder named `<original_table>`** at an appropriate point in the text to indicate where the table would appear. Do not reproduce the table’s contents.\n\n### Hard constraints (non-negotiable)\n\n* The Excel table is the only factual data source.\n* Introduce the data **naturally**, without explicitly labeling it as a dataset, source file, or input.\n* Do not describe how the data was inspected, reviewed, or processed.\n* All factual claims must be consistent with the table.\n* Write report in Korean\n\n### <original_table> placement rule (critical)\n* Include exactly one <original_table> placeholder.\n* Embed <original_table> as part of the natural flow of text.\n* Do not place it on its own section.\n* The surrounding sentence must read naturally if the placeholder were

In [10]:
report_prompt = report_generator_prompts + f'''=======INPUT MARKDOWN======{md_all}'''
json_prompt = json_generator_prompts + f'''=======INPUT MARKDOWN======{md_all}'''

In [11]:
report_prompt

'You are given an Markdown that parsed from Excel spreadsheet.\n\nYour task is to write a **professional analytical report** grounded in the data, written as a coherent standalone document.\n\nYou **must include a placeholder named `<original_table>`** at an appropriate point in the text to indicate where the table would appear. Do not reproduce the table’s contents.\n\n### Hard constraints (non-negotiable)\n\n* The Excel table is the only factual data source.\n* Introduce the data **naturally**, without explicitly labeling it as a dataset, source file, or input.\n* Do not describe how the data was inspected, reviewed, or processed.\n* All factual claims must be consistent with the table.\n* Write report in Korean\n\n### <original_table> placement rule (critical)\n* Include exactly one <original_table> placeholder.\n* Embed <original_table> as part of the natural flow of text.\n* Do not place it on its own section.\n* The surrounding sentence must read naturally if the placeholder were

In [12]:
json_prompt

'You will be given structured extracts derived from a source document.\nYour task is to produce two artifacts from this information:\n\n1. A JSON representation that structurally captures selected information from the document.\n2. A JSON Schema that describes exactly and only the structure of the generated JSON.\n\nIMPORTANT WORLDVIEW CONSTRAINTS:\n\n* You must NOT assume that the source data comes from spreadsheets, tables, cells, or files of any kind.\n* The JSON must be fully derivable from the document text alone.\n* The JSON Schema must be specific to this JSON output and MAY differ for every document.\n* If a concept does not explicitly appear in the document, it must NOT appear in the JSON or the JSON Schema.\n* Do NOT introduce new facts, interpretations, entities, inferred classifications, or additional categories beyond what is explicitly stated.\n* If information is missing, do not fabricate values. Omit the field entirely unless the absence itself is explicitly stated in t

In [13]:
import os
from google import genai

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])  
resp_report = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=report_prompt
)

In [14]:
import os
from google import genai

client = genai.Client(api_key=os.environ["GEMINI_API_KEY"])  
resp_jsons = client.models.generate_content(
    model="gemini-2.0-flash",
    contents=json_prompt
)

In [15]:
resp_report.text

p = Path(f'../data/debug.txt')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = resp_report.text.replace("\r\n", "\n")
p.write_text(normalized, encoding="utf-8")

1155

In [16]:
from utils.parsing_answer import parse_json_and_schema ,replace_original_table_in_report

data = parse_json_and_schema(resp_jsons.text)
report = replace_original_table_in_report(resp_report.text, md_all)

In [17]:
report

"금번 학기 폐강 확정 강좌 현황을 분석한 결과, 교양 과목과 교직 과목에서 폐강이 집중적으로 발생한 것으로 나타났다. 특히 교양 과목의 경우, '인물로본중국사' (수강인원 14명), '지방자치의이해' (수강인원 4명), '피드백을통한논문작성의실제' (수강인원 12명) 등이 폐강 기준 인원을 충족하지 못해 폐강되었다. 교직 과목 또한 '교육학개론' (수강인원 11명) 및 '교육과정' (수강인원 4명)이 낮은 수강 인원으로 인해 폐강 처리되었다. 일반선택 과목 중에서는 '소통-행복과변화로가는길'이 30명의 수강생을 확보했음에도 폐강 기준인원 60명에 미달하여 폐강된 점이 눈에 띈다.\n\n이러한 폐강 현황은 학생들의 수강 선택 패턴, 과목의 매력도, 그리고 폐강 기준 인원 설정의 적절성 등 다양한 요인에 의해 영향을 받는 것으로 보인다. 예를 들어, 특정 학과의 학생들이 특정 교양 과목을 수강할 수 없는 제한 조건이 있는 경우, 해당 과목의 수강 인원이 감소하여 폐강으로 이어질 수 있다. 실제, 앞서 언급된 '인물로본중국사'와 '지방자치의이해' 과목의 경우, 각각 역사학과와 행정학과/새마을국제개발학과 학생들의 수강이 제한되어 있었다. 다음 문장은 폐강 과목 목록에 대한 추가적인 정보를 제공하는데, 좀 더 자세한 사항은 다음 표에서 확인할 수 있다. ## Sheet: 폐강확정강좌(게시용)\n\n|   폐강기준 인원 |   수강인원 (대학원포함) | 승인원  검토 결과   |   학년 | 과정구분   |   수강번호 | 과목명                     |   학점 | 담당교수                  | 개설대학   | 개설학과   | 주관대학     | 주관학과   | 수업형태   | 비고                                   | 강의타입   |\n|----------------:|------------------------:|:--------------------|-------:|:-----------|-----------

In [18]:
p = Path(f'../data/report/{FILE_NAME}.txt')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = report.replace("\r\n", "\n")
p.write_text(normalized, encoding="utf-8")

import json

p = Path(f'../data/json/{FILE_NAME}.json')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = data["json_obj"]
p.write_text(json.dumps(normalized, ensure_ascii=False, indent=2), encoding="utf-8")


p = Path(f'../data/json_schema/{FILE_NAME}.json')

p.parent.mkdir(parents=True, exist_ok=True)
normalized = data["json_schema"]
p.write_text(json.dumps(normalized, ensure_ascii=False, indent=2), encoding="utf-8")

1612

In [19]:
data["json_obj"]

{'강좌': [{'폐강기준인원': 60,
   '수강인원': 30,
   '승인원검토결과': '폐강',
   '학년': 1,
   '과정구분': '일반선택',
   '수강번호': 319,
   '과목명': '소통-행복과변화로가는길',
   '학점': 3,
   '담당교수': '이윤주',
   '개설대학': '대학',
   '개설학과': '대학',
   '주관대학': '사범대학',
   '주관학과': '교육학과',
   '수업형태': '인터넷강의',
   '비고': '인터넷강의',
   '강의타입': '일반강의'},
  {'폐강기준인원': 20,
   '수강인원': 11,
   '승인원검토결과': '폐강',
   '학년': 2,
   '과정구분': '교직',
   '수강번호': 363,
   '과목명': '교육학개론',
   '학점': 2,
   '담당교수': '이상린',
   '개설대학': '대학',
   '개설학과': '대학',
   '주관대학': '사범대학',
   '주관학과': '교육학과',
   '강의타입': '일반강의'},
  {'폐강기준인원': 20,
   '수강인원': 4,
   '승인원검토결과': '폐강',
   '학년': 3,
   '과정구분': '교직',
   '수강번호': 2407,
   '과목명': '교육과정',
   '학점': 2,
   '담당교수': '배지현',
   '개설대학': '사범대학',
   '개설학과': '한문교육과',
   '주관대학': '사범대학',
   '주관학과': '교육학과',
   '강의타입': '일반강의'},
  {'폐강기준인원': 20,
   '수강인원': 14,
   '승인원검토결과': '폐강',
   '학년': 1,
   '과정구분': '교양',
   '수강번호': 214,
   '과목명': '인물로본중국사',
   '학점': 3,
   '담당교수': '류준형',
   '개설대학': '교양학부',
   '개설학과': '교양학부',
   '주관대학': '인문대학',
   '주관학과': '역사학과',
   '비

In [20]:
data["json_schema"]

{'$schema': 'https://json-schema.org/draft/2020-12/schema',
 'type': 'object',
 'properties': {'강좌': {'type': 'array',
   'items': {'type': 'object',
    'properties': {'폐강기준인원': {'type': 'integer'},
     '수강인원': {'type': 'integer'},
     '승인원검토결과': {'type': 'string'},
     '학년': {'type': 'integer'},
     '과정구분': {'type': 'string'},
     '수강번호': {'type': 'integer'},
     '과목명': {'type': 'string'},
     '학점': {'type': 'integer'},
     '담당교수': {'type': 'string'},
     '개설대학': {'type': 'string'},
     '개설학과': {'type': 'string'},
     '주관대학': {'type': 'string'},
     '주관학과': {'type': 'string'},
     '수업형태': {'type': 'string'},
     '비고': {'type': 'string'},
     '강의타입': {'type': 'string'}},
    'required': ['폐강기준인원',
     '수강인원',
     '승인원검토결과',
     '학년',
     '과정구분',
     '수강번호',
     '과목명',
     '학점',
     '담당교수',
     '개설대학',
     '개설학과',
     '주관대학',
     '주관학과',
     '강의타입'],
    'additionalProperties': False}}},
 'required': ['강좌'],
 'additionalProperties': False}